# Controlling the measuring instrument 'Yokogawa GP10' by web interface

In [1]:
# import necessary libraries
import urllib as http
import pandas as pd

In [2]:
ip_of_gp10 = '192.168.10.207'

## Recording data to SD card

### Set the file type of recorded file to 'Text'

With this setting, ASCII text files are written to the SD card. These can be read in as CSV files in common spreadsheet programs. A tabulator is used as column separator.

**Attention:** This file format is NOT readable in the tool *Universal Viewer*! Therefore, for other colleagues, the format must be changed back to "Binary" at the end of the measurement!

In [3]:
# URI for setting the file type
file_type = "Text"
str_uri = "http://" + ip_of_gp10 + "/gene.cgi?msg=SDirectory,'DATA0';SFileHead,'';SFileName,Date,'';SMediaSave,On,Off;SFileFormat," + file_type

str_uri

"http://192.168.15.207/gene.cgi?msg=SDirectory,'DATA0';SFileHead,'';SFileName,Date,'';SMediaSave,On,Off;SFileFormat,Text"

In [4]:
# Set the file type by http GET request
str_response = http.request.urlopen(str_uri).read().decode('utf-8')
str_response = str_response.replace('\r\n', '')

if (str_response == 'E0'):
    print("Setting file type was successful: {}".format(str_response))
else:
    print("There was an error while setting the file type: {}".format(str_response))

Setting file type was sucessful: E0


## Configure the measurement recording channels

In [62]:
# Helper function for building the config URI and write the configurations
def config_meas_rec_channel(str_channel, str_name, str_color):
    str_uri = "http://" + ip_of_gp10 + "/gene.cgi?msg=STagIO," + str_channel + ",'" + str_name + "','';" \
               "SColorIO," + str_channel + "," + str_color + ";" \
               "SZoneIO," + str_channel + ",0,100;" \
               "SScaleIO," + str_channel + ",1,10;" \
               "SBarIO," + str_channel + ",Lower,10;" \
               "SBandIO," + str_channel + ",Off,102,255,0,0,100;" \
               "SAlmMarkIO," + str_channel + ",On,Alarm,255,0,0,255,153,51,255,153,51,255,0,0;" \
               "SValueIO," + str_channel + ",'',''"
    
    str_response = http.request.urlopen(str_uri).read().decode('utf-8')
    str_response = str_response.replace('\r\n', '')

    if (str_response == 'E0'):
        print("Configuring the channel was successful: {}".format(str_response))
    else:
        print("Configuring the channel was NOT successful: {}".format(str_response))
    
    return str_uri, str_response

In [63]:
# Dataframe with the config values for the measuring channels
df_meas_rec_channels = pd.DataFrame(columns=['Channel', 'Track Name', 'Color RGB'],
                                    data=[['0001', 'T01_Trafo_Gehaeuse', '255,0,0'],
                                          ['0002', 'T02_Gehaeuse_innen', '0,153,51'],
                                          ['0003', 'T03_Gehaeuse_hinten', '0,51,255'],
                                          ['0004', 'T04_Kaffetasse', '204,102,204'],
                                          ['0005', 'T05_Umgebung', '119,170,221'],
                                         ])


display(df_meas_rec_channels)

,Channel,Track Name,Color RGB
0,0001,T01_Trafo_Gehaeuse,"255,0,0"
1,0002,T02_Gehaeuse_innen,"0,153,51"
2,0003,T03_Gehaeuse_hinten,"0,51,255"
3,0004,T04_Kaffetasse,"204,102,204"
4,0005,T05_Umgebung,"119,170,221"


In [65]:
# Iterate the config dataframe and upload the configurations
for index, row in df_meas_rec_channels.iterrows():
    print("Channel: {}, Track Name: {}, Color RGB: {}\r\n".format(row['Channel'], row['Track Name'], row['Color RGB']))
    
    str_uri, str_response = config_meas_rec_channel(row['Channel'], row['Track Name'], row['Color RGB'])

    print("URI: {}\r\n Response: {}\r\n".format(str_uri, str_response))

Channel: 0001, Track Name: T01_Trafo_Gehaeuse, Color RGB: 255,0,0

Configuring the channel was sucessful: E0
URI: http://192.168.15.207/gene.cgi?msg=STagIO,0001,'T01_Trafo_Gehaeuse','';SColorIO,0001,255,0,0;SZoneIO,0001,0,100;SScaleIO,0001,1,10;SBarIO,0001,Lower,10;SBandIO,0001,Off,102,255,0,0,100;SAlmMarkIO,0001,On,Alarm,255,0,0,255,153,51,255,153,51,255,0,0;SValueIO,0001,'',''
 Response: E0

Channel: 0002, Track Name: T02_Gehaeuse_innen, Color RGB: 0,153,51

Configuring the channel was sucessful: E0
URI: http://192.168.15.207/gene.cgi?msg=STagIO,0002,'T02_Gehaeuse_innen','';SColorIO,0002,0,153,51;SZoneIO,0002,0,100;SScaleIO,0002,1,10;SBarIO,0002,Lower,10;SBandIO,0002,Off,102,255,0,0,100;SAlmMarkIO,0002,On,Alarm,255,0,0,255,153,51,255,153,51,255,0,0;SValueIO,0002,'',''
 Response: E0

Channel: 0003, Track Name: T03_Gehaeuse_hinten, Color RGB: 0,51,255

Configuring the channel was sucessful: E0
URI: http://192.168.15.207/gene.cgi?msg=STagIO,0003,'T03_Gehaeuse_hinten','';SColorIO,0003,0,

## Start the recording

In [5]:
# URI for starting (0) the recording of measurements
start_stop = "0"
str_uri = "http://" + ip_of_gp10 + "/gene.cgi?msg=ORec," + start_stop

str_uri

'http://192.168.15.207/gene.cgi?msg=ORec,0'

In [6]:
# Start the recording
str_response = http.request.urlopen(str_uri).read().decode('utf-8')
str_response = str_response.replace('\r\n', '')

if (str_response == 'E0'):
    print("Start the recording was successful: {}".format(str_response))
else:
    print("There was an error: {}".format(str_response))

Start the recording was sucessful: E0


## Stop the recording

In [ ]:
# URI for stopping (1) the recording of measurements
start_stop = "1"
str_uri = "http://" + ip_of_gp10 + "/gene.cgi?msg=ORec," + start_stop

str_uri

In [ ]:
# Stop the recording
str_response = http.request.urlopen(str_uri).read().decode('utf-8')
str_response = str_response.replace('\r\n', '')

if (str_response == 'E0'):
    print("Stop the recording was successful: {}".format(str_response))
else:
    print("There was an error: {}".format(str_response))

### Set the file type of recorded file to 'Binary'

This format is the default setting of the measuring device. The resulting **GDS** files can be further processed by the Windows tool *Universal Viewer*.

The specification of the GDS file format is kept secret after inquiry with the Yokogawa support -- for "security" reasons was said there ... But this is quite nonsense, because "security by obscurity" is not a good security approach, as several incidents have impressively shown ...

In [22]:
# URI for setting the file type
file_type = "Binary"
str_uri = "http://" + ip_of_gp10 + "/gene.cgi?msg=SDirectory,'DATA0';SFileHead,'';SFileName,Date,'';SMediaSave,On,Off;SFileFormat," + file_type

str_uri

"http://192.168.15.207/gene.cgi?msg=SDirectory,'DATA0';SFileHead,'';SFileName,Date,'';SMediaSave,On,Off;SFileFormat,Binary"

In [23]:
# Set the file type by http GET request
str_response = http.request.urlopen(str_uri).read().decode('utf-8')
str_response = str_response.replace('\r\n', '')

if (str_response == 'E0'):
    print("Setting file type was successful: {}".format(str_response))
else:
    print("There was an error while setting the file type: {}".format(str_response))

Setting file type was sucessful: E0


## Read in the file list from SD card

In [3]:
# URI for getting the file list
str_uri = 'http://' + ip_of_gp10 + '/gene.cgi?msg=FMedia,DIR,%27/DRV0/DATA0/%27,1'

# Convert byte stream from the http response to string 
str_file_list = http.request.urlopen(str_uri).read().decode('utf-8')
str_file_list

'EA\r\n20/12/09 12:47:34      <DIR> .\r\n20/12/09 12:47:34      <DIR> ..\r\n20/12/09 12:47:34     424432 000086_201209_080310.GDS\r\n21/03/18 15:35:58      15328 000087_210318_153110.GDS\r\n21/03/18 15:45:14      12640 000088_210318_154210.GDS\r\n21/03/18 16:13:44       8984 000089_210318_161310.GDS\r\n21/03/18 16:27:08      25816 000090_210318_161510.GDS\r\n21/03/19 09:50:04     110240 000091_210319_084020.GDS\r\n21/03/25 16:59:18      19440 000092_210325_165150.GDS\r\n21/03/26 16:59:00     430832 000093_210326_121020.GDS\r\n21/03/31 18:48:44     597960 000094_210331_120550.GDS\r\n21/04/01 16:48:50     425936 000095_210401_120330.GDS\r\n21/04/06 17:15:20     277344 000096_210406_141130.GDS\r\n21/04/07 16:02:48     559048 000094_210331_120550.GDS.vdx\r\n21/04/07 16:06:02     559048 000095_210401_120330.GDS.vdx\r\n21/04/07 16:09:18     559048 000096_210406_141130.GDS.vdx\r\n21/04/08 16:52:34     460344 000097_210408_114340.GDS\r\n21/04/15 09:26:40       5850 000098_210415_092340.GTD\r\n

In [4]:
# Split string by '\r\n' and convert it to a list
lst_file_list = list(str_file_list.split('\r\n'))
lst_file_list

['EA',
 '20/12/09 12:47:34      <DIR> .',
 '20/12/09 12:47:34      <DIR> ..',
 '20/12/09 12:47:34     424432 000086_201209_080310.GDS',
 '21/03/18 15:35:58      15328 000087_210318_153110.GDS',
 '21/03/18 15:45:14      12640 000088_210318_154210.GDS',
 '21/03/18 16:13:44       8984 000089_210318_161310.GDS',
 '21/03/18 16:27:08      25816 000090_210318_161510.GDS',
 '21/03/19 09:50:04     110240 000091_210319_084020.GDS',
 '21/03/25 16:59:18      19440 000092_210325_165150.GDS',
 '21/03/26 16:59:00     430832 000093_210326_121020.GDS',
 '21/03/31 18:48:44     597960 000094_210331_120550.GDS',
 '21/04/01 16:48:50     425936 000095_210401_120330.GDS',
 '21/04/06 17:15:20     277344 000096_210406_141130.GDS',
 '21/04/07 16:02:48     559048 000094_210331_120550.GDS.vdx',
 '21/04/07 16:06:02     559048 000095_210401_120330.GDS.vdx',
 '21/04/07 16:09:18     559048 000096_210406_141130.GDS.vdx',
 '21/04/08 16:52:34     460344 000097_210408_114340.GDS',
 '21/04/15 09:26:40       5850 000098_21

In [5]:
# Read file list to dataframe
df_file_list = pd.DataFrame(lst_file_list)

# Drop first 3 and last 2 rows
df_file_list.drop([0, 1, 2], inplace=True)
df_file_list.drop(df_file_list.tail(2).index, inplace=True)

# Split single column into multiple ones
df_file_list_split = df_file_list[0].str.split(expand=True)

# Reset the index of dataframe and avoid the old index being added as a column
df_file_list_split.reset_index(drop=True, inplace=True)

In [6]:
df_file_list_split.head(3)

,0,1,2,3
0,20/12/09,12:47:34,424432,000086_201209_080310.GDS
1,21/03/18,15:35:58,15328,000087_210318_153110.GDS
2,21/03/18,15:45:14,12640,000088_210318_154210.GDS


In [7]:
df_file_list_split.tail(3)

,0,1,2,3
26,21/05/04,19:18:48,5445,000108_210504_191610.GTD
27,21/05/05,12:13:50,117697,000109_210505_104320.GTD
28,21/05/05,12:17:14,1555,000110_210505_121620.GTD


In [8]:
# Rename columns by column index
df_file_list_new = df_file_list_split.rename(columns={ df_file_list_split.columns[0]: "Date", 
                                                       df_file_list_split.columns[1]: "Time",
                                                       df_file_list_split.columns[2]: "File size",
                                                       df_file_list_split.columns[3]: "File name" })

df_file_list_new.head(3)

,Date,Time,File size,File name
0,20/12/09,12:47:34,424432,000086_201209_080310.GDS
1,21/03/18,15:35:58,15328,000087_210318_153110.GDS
2,21/03/18,15:45:14,12640,000088_210318_154210.GDS


In [9]:
# Combine date and time columns to a new datetime array
# Hints:
# Parameter 'format="%y/%m/%d %H:%M:%S"' defines the INPUT data format of the raw original string
# With method '.dt.strftime('%d.%m.%Y, %H:%M:%S')' you can define the OUTPUT data format
new_dt_column = pd.to_datetime(df_file_list_new['Date'] + ' ' + df_file_list_new['Time'], format="%y/%m/%d %H:%M:%S") #.dt.strftime('%d.%m.%Y, %H:%M:%S')

# Replace values of 'Date' row with new datetime array
df_file_list_new['Date'] = new_dt_column

# Rename row to 'Datetime'
df_file_list_new = df_file_list_new.rename(columns={'Date': 'Datetime'})

# Delete unused row 'Time'
del df_file_list_new['Time']

# Cast values to correct data types
df_file_list_new['File size'] = df_file_list_new['File size'].astype(int)
df_file_list_new['File name'] = df_file_list_new['File name'].astype(str)

In [10]:
# View first 5 files in the list
df_file_list_new.head(5)

,Datetime,File size,File name
0,2020-12-09 12:47:34,424432,000086_201209_080310.GDS
1,2021-03-18 15:35:58,15328,000087_210318_153110.GDS
2,2021-03-18 15:45:14,12640,000088_210318_154210.GDS
3,2021-03-18 16:13:44,8984,000089_210318_161310.GDS
4,2021-03-18 16:27:08,25816,000090_210318_161510.GDS


In [11]:
df_file_list_new.dtypes

Datetime     datetime64[ns]
File size             int32
File name            object
dtype: object

In [12]:
# Delete rows with file types other than GTD
# '~' negates the search string
df_file_list_new.drop(df_file_list_new[~df_file_list_new['File name'].str.contains("GTD")].index, inplace=True)

# Reset the index of dataframe and avoid the old index being added as a column
df_file_list_new.reset_index(drop=True, inplace=True)

In [13]:
# Sort the dataframe by column 'Datetime' descending to get the newest file on top
df_file_list_new.sort_values(by='Datetime', ascending=False, inplace=True)

In [14]:
display(df_file_list_new)

,Datetime,File size,File name
6,2021-05-05 12:17:14,1555,000110_210505_121620.GTD
5,2021-05-05 12:13:50,117697,000109_210505_104320.GTD
4,2021-05-04 19:18:48,5445,000108_210504_191610.GTD
3,2021-05-04 15:26:24,3740,000107_210504_152500.GTD
2,2021-05-04 14:04:44,26320,000100_210504_134540.GTD
1,2021-05-04 13:41:48,2229,000099_210504_134140.GTD
0,2021-04-15 09:26:40,5850,000098_210415_092340.GTD


## Download GDS/GTD files to local file system

Source: https://stackabuse.com/download-files-with-python/

In [21]:
# Get name of last file in dataframe
#str_file_name = df_file_list_new['File name'].iloc[-1]

# Get the name of the first file in dataframe
# It should be the newest file sorted by descending order
str_file_name = df_file_list_new['File name'].iloc[0]

str_file_name

'000110_210505_121620.GTD'

In [19]:
# Expand the download URI by file name
file_url = 'http://' + ip_of_gp10 + '/DRV0/DATA0/'
file_url = file_url + str_file_name

file_url

'http://192.168.15.207/DRV0/DATA0/000109_210505_104320.GTD'

In [20]:
# Download the file to local file system
http.request.urlretrieve(file_url, './data_files/' + str_file_name)

('./data_files/000109_210505_104320.GTD',
 <http.client.HTTPMessage at 0x6897f2b0>)